In [1]:
%load_ext autoreload
%autoreload 2

from src.model.news_encoder import LNEConfig
from src.model.user_encoder import CPJAConfig, PEConfig,NSAConfig,PAUEConfig
from src.model.popularity_predictor import TANPPConfig, CRGConfig, CBPDConfig, RBPDConfig, REConfig
from src.model.pprec import PPRConfig, PAGConfig, PPRec

from src.data.split import EBNeRDSplit

import torch

Some data for testing


In [2]:
split = EBNeRDSplit()

Configuration of the model

In [ ]:
# This depeneds on how were going to make the dataloader
max_clicked_articles_in_history = 50

# This is the config we can do experiments with.
# Any configuration should work. For instance
# size_n in the user news encoder doesnt have to
# match the one in the popularity news encoder.
# I filled it in the way they implemented it.
# Apart from the news encoder of course, which
# is still using the embeddings from the artifacts.
pprec_config = PPRConfig(
    user_news_encoder_config=LNEConfig(size_n=400, model="bert"),
    popularity_news_encoder_config=LNEConfig(size_n=400, model="bert"),
    user_encoder_config=PAUEConfig(
        popularity_embedding_config=PEConfig(
            size_p=100,
            max_ctr=200,
        ),
        news_self_attention_config=NSAConfig(
            n_attention_heads=20,
            head_output_size=20,
        ),
        content_popularity_joint_attention_config=CPJAConfig(weight_size=100),
    ),
    popularity_predictor_config=TANPPConfig(
        recency_based_popularity_dense_config=RBPDConfig(),
        content_based_popularity_dense_config=CBPDConfig(),
        recency_embedding_config=REConfig(
            r_size=100,
            max_recency=1500,
        ),
        content_recency_gate_config=CRGConfig(),
    ),
    aggregator_gate_config=PAGConfig(),
)

In [ ]:
model = PPRec(
    max_clicked=max_clicked_articles_in_history,
    device=torch.device("cpu"),
    config=pprec_config,
)